In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp, norm

#############################################
# 1) Manually specify your CSV file paths.
#    For example, let's say you have:
#    - mouse1_patch.csv, mouse2_patch.csv
#    - mouse1_interpatch.csv, mouse2_interpatch.csv
#    Adjust these to your actual files.
#############################################
patch_csvs = [
    "./99/left_bottom/patch_RSC.csv",
    "./118/left_bottom/patch_RSC.csv",
    "./80/left_bottom/patch_RSC.csv",
    "./82/left_bottom/patch_RSC.csv"
    # Add more if you have them
]
interpatch_csvs = [
    "./99/left_bottom/interpatch_RSC.csv",
    "./118/left_bottom/interpatch_RSC.csv",
    "./80/left_bottom/interpatch_RSC.csv",
    "./82/left_bottom/interpatch_RSC.csv"
    # Add more if you have them
]
#############################################
# 2) A helper function to load and return (bin_starts, counts)
#############################################
def load_histogram(csv_path):
    """
    Reads a CSV with columns:
        index, bin start, count
    Returns two numpy arrays:
        bin_starts, counts
    """
    df = pd.read_csv(csv_path)
    return df["bin start"].values, df["count"].values

#############################################
# 3) Aggregate (sum) counts across all "patch" CSVs and all "interpatch" CSVs
#############################################
accum_patch_counts = None
accum_interpatch_counts = None
bin_starts = None

# --- Combine patch CSVs ---
for fp in patch_csvs:
    b_starts, c_vals = load_histogram(fp)
    
    # Keep track of bin_starts the first time
    if bin_starts is None:
        bin_starts = b_starts
        accum_patch_counts = c_vals.copy()  # Start from this file
    else:
        # Make sure bin_starts match across CSVs; if not, you'd need to interpolate
        assert np.allclose(bin_starts, b_starts), "Bin edges mismatch!"
        accum_patch_counts += c_vals

# --- Combine interpatch CSVs ---
for fp in interpatch_csvs:
    b_starts, c_vals = load_histogram(fp)
    # bin_starts should match the ones from patch
    assert np.allclose(bin_starts, b_starts), "Bin edges mismatch!"
    if accum_interpatch_counts is None:
        accum_interpatch_counts = c_vals.copy()
    else:
        accum_interpatch_counts += c_vals

#############################################
# 4) (Optional) Convert to probabilities (so total area sums to 1)
#    This helps compare distributions across different total pixel counts.
#############################################
sum_patch = accum_patch_counts.sum()
sum_interpatch = accum_interpatch_counts.sum()

patch_prob = accum_patch_counts / sum_patch
interpatch_prob = accum_interpatch_counts / sum_interpatch

#############################################
# 5) Define bin centers (for plotting) by adding half of the bin width
#############################################
if len(bin_starts) > 1:
    bin_width = bin_starts[1] - bin_starts[0]
else:
    bin_width = 1.0  # fallback if only 1 bin
bin_centers = bin_starts + (bin_width / 2.0)

#############################################
# 6) Plot the aggregated histograms (like Figure L)
#############################################
plt.figure(figsize=(6,4))

# Patch in black bars
plt.bar(
    bin_centers, 
    patch_prob, 
    width=bin_width, 
    alpha=0.5, 
    color='black', 
    label='Patch'
)

# Interpatch in gray bars
plt.bar(
    bin_centers, 
    interpatch_prob, 
    width=bin_width, 
    alpha=0.5, 
    color='gray', 
    label='Interpatch'
)

plt.xlabel("Intensity (arbitrary units)")
plt.ylabel("Normalized Frequency")
plt.legend()

#############################################
# 7) (Optional) Fit each distribution to a Gaussian and overplot
#############################################
# Reconstruct intensities from the bin centers and counts:
#   we'll expand each bin center repeated by 'counts' times
#   to get the approximate distribution of intensities.
intensities_patch = np.repeat(bin_centers, accum_patch_counts.astype(int))
intensities_interpatch = np.repeat(bin_centers, accum_interpatch_counts.astype(int))

from scipy.stats import norm

# Fit Gaussian
mu_patch, sigma_patch = norm.fit(intensities_patch)
mu_inter, sigma_inter = norm.fit(intensities_interpatch)

# We'll define a range for x-values
x = np.linspace(bin_centers[0], bin_centers[-1], 200)
pdf_patch = norm.pdf(x, mu_patch, sigma_patch)
# Scale the PDF to match histogram height
pdf_patch_scaled = pdf_patch * len(intensities_patch) * (bin_width / sum_patch)

pdf_inter = norm.pdf(x, mu_inter, sigma_inter)
pdf_inter_scaled = pdf_inter * len(intensities_interpatch) * (bin_width / sum_interpatch)

plt.plot(x, pdf_patch_scaled, 'k-', lw=2)
plt.plot(x, pdf_inter_scaled, 'gray', lw=2)

#############################################
# 8) (Optional) Perform a KS test
#############################################
from scipy.stats import ks_2samp
ks_stat, p_value = ks_2samp(intensities_patch, intensities_interpatch)
plt.title(f"KS p={p_value:.3e}")

plt.tight_layout()
plt.show()


AssertionError: Bin edges mismatch!